In [48]:
import pandas as pd
from torchtext.data import Field
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import spacy
import re
import pickle
from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint
from torchtext import data
from torchtext import datasets

from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
from torchtext.datasets import TranslationDataset
from torchtext.data import Field, BucketIterator

In [17]:
df = pd.read_csv('./data_torch/pmindia.v1.hi-en.csv')

In [18]:
df.columns

Index(['english_sentence', 'hindi_sentence'], dtype='object')

In [49]:
spacy_eng = spacy.load("en_core_web_sm")

str_punct = '''[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~।]'''
def tokenize_hi(text):
    return [re.sub(str_punct,'',tok).lower() for tok in tokenize(text, "hi")]


def tokenize_eng(text):
    return [re.sub(str_punct,'',tok.text).lower() for tok in spacy_eng.tokenizer(text)]


In [52]:
print

<function print>

In [53]:
# text_field = Field(
#     tokenize='basic_english', 
#     lower=True
# )

text_field = Field(
    tokenize=tokenize_eng, 
    lower=True
)

In [54]:
label_field = Field(sequential=False, use_vocab=False)
# sadly have to apply preprocess manually
preprocessed_text = df['english_sentence'].apply(lambda x: text_field.preprocess(x))

In [55]:
len(preprocessed_text)

56831

In [68]:
# load fastext simple embedding with 300d
text_field.build_vocab(
    preprocessed_text, 
    vectors='glove.6B.300d',max_size=17000, min_freq=2
)
# get the vocab instance
vocab = text_field.vocab

In [73]:
dict_glove = vocab.itos

In [84]:
# [i for i in dict_glove if type(i) is not float  and type(int(i)) == int]

In [70]:
dir(vocab)

['UNK',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_default_unk_index',
 'extend',
 'freqs',
 'itos',
 'load_vectors',
 'lookup_indices',
 'set_vectors',
 'stoi',
 'unk_index',
 'vectors']

In [69]:
vocab.vectors.shape

torch.Size([17002, 300])

In [46]:
vocab.vectors[-10001].shape

torch.Size([300])

In [8]:
from torchtext.vocab import GloVe
embedding_glove = GloVe(name='6B', dim=300)

In [9]:
# known token, in my case print 12
print(vocab['are'])
# unknown token, will print 0
print(vocab['crazy'])

12
0


In [15]:
from torchtext.data import Field
text_field = Field(
    sequential=True,
    tokenize='basic_english', 
    fix_length=5,
    lower=True
)
label_field = Field(sequential=False, use_vocab=False)
# sadly have to apply preprocess manually
preprocessed_text = df['text'].apply(
    lambda x: text_field.preprocess(x)
)
# load fastext simple embedding with 300d
text_field.build_vocab(
    preprocessed_text, 
    vectors='glove.6B.300d'
)
# get the vocab instance
vocab = text_field.vocab

/anaconda/envs/inltk/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [16]:
vocab.vectors.shape

torch.Size([18, 300])

In [13]:
x = 'The news-test2011 set has three additional Czech translations that you may want to use. You can download them from Charles University.'

In [1]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import spacy
import re
import pickle
from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint
from torchtext import data
from torchtext import datasets

from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
from torchtext.datasets import TranslationDataset
from torchtext.data import Field, BucketIterator

In [2]:
from inltk.inltk import tokenize

In [65]:
# spacy_eng = spacy.load("en_core_web_sm")

# str_punct = '''[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~।]'''
# def tokenize_hi(text):
#     return [re.sub(str_punct,'',tok).lower() for tok in tokenize(text, "hi")]


# def tokenize_eng(text):
#     return [tok.text for tok in spacy_eng.tokenizer(text)]


# hindi = Field(tokenize=tokenize_hi, lower=True, init_token="<sos>", eos_token="<eos>")

# english = Field(
#     tokenize=tokenize_eng, lower=True, init_token="<sos>", eos_token="<eos>"
# )

# train_data = TranslationDataset('./data_torch/data_sm',
#     exts=(".hi", ".en"), fields=(hindi, english)
# )

# hindi.build_vocab(train_data, max_size=10000, min_freq=2)
# english.build_vocab(train_data, max_size=10000, min_freq=2)

In [3]:
spacy_eng = spacy.load("en_core_web_sm")

str_punct = '''[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~।]'''
def tokenize_hi(text):
    return [re.sub(str_punct,'',tok).lower() for tok in tokenize(text, "hi")]


def tokenize_eng(text):
    return [re.sub(str_punct,'',tok.text).lower() for tok in spacy_eng.tokenizer(text)]


In [4]:
hindi = Field(tokenize=tokenize_hi, lower=True, init_token="<sos>", eos_token="<eos>")

english = Field(
    tokenize=tokenize_eng, lower=True, init_token="<sos>", eos_token="<eos>"
)


In [5]:
hindi = data.Field(tokenize=tokenize_hi)
english = data.Field(tokenize=tokenize_eng)
mt_train = datasets.TranslationDataset(
     path='./data_torch/data_sm', exts=('.hi', '.en'),
     fields=(hindi, english))
hindi.build_vocab(mt_train, max_size=15000, min_freq=2)
english.build_vocab(mt_train, max_size=15000, min_freq=2)


In [6]:
# with open('mt_train.pickle', 'wb') as handle:
#     pickle.dump(mt_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
# with open('src.pickle', 'rb') as handle:
#     src_loaded = pickle.load(handle)

# with open('trg.pickle', 'rb') as handle:
#     trg_loaded = pickle.load(handle)

In [8]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, bidirectional=True)

        self.fc_hidden = nn.Linear(hidden_size * 2, hidden_size)
        self.fc_cell = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        # x: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        encoder_states, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)

        # Use forward, backward cells and hidden through a linear layer
        # so that it can be input to the decoder which is not bidirectional
        # Also using index slicing ([idx:idx+1]) to keep the dimension
        hidden = self.fc_hidden(torch.cat((hidden[0:1], hidden[1:2]), dim=2))
        cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))

        return encoder_states, hidden, cell


class Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p
    ):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(hidden_size * 2 + embedding_size, hidden_size, num_layers)

        self.energy = nn.Linear(hidden_size * 3, 1)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()

    def forward(self, x, encoder_states, hidden, cell):
        x = x.unsqueeze(0)
        # x: (1, N) where N is the batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        sequence_length = encoder_states.shape[0]
        h_reshaped = hidden.repeat(sequence_length, 1, 1)
        # h_reshaped: (seq_length, N, hidden_size*2)

        energy = self.relu(self.energy(torch.cat((h_reshaped, encoder_states), dim=2)))
        # energy: (seq_length, N, 1)

        attention = self.softmax(energy)
        # attention: (seq_length, N, 1)

        # attention: (seq_length, N, 1), snk
        # encoder_states: (seq_length, N, hidden_size*2), snl
        # we want context_vector: (1, N, hidden_size*2), i.e knl
        context_vector = torch.einsum("snk,snl->knl", attention, encoder_states)

        rnn_input = torch.cat((context_vector, embedding), dim=2)
        # rnn_input: (1, N, hidden_size*2 + embedding_size)

        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs).squeeze(0)
        # predictions: (N, hidden_size)

        return predictions, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(english.vocab)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        encoder_states, hidden, cell = self.encoder(source)

        # First input will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # At every time step use encoder_states and update hidden, cell
            output, hidden, cell = self.decoder(x, encoder_states, hidden, cell)

            # Store prediction for current time step
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = False
save_model = True

In [10]:
# Training hyperparameters
num_epochs = 200
learning_rate = 3e-4
batch_size = 32

In [11]:
# Model hyperparameters
input_size_encoder = len(hindi.vocab)
input_size_decoder = len(english.vocab)
output_size = len(english.vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 1
enc_dropout = 0.0
dec_dropout = 0.0

In [12]:
# Tensorboard to get nice loss plot
writer = SummaryWriter(f"runs/loss_plot")
step = 0

In [13]:
# train_iterator = BucketIterator.splits(
#     (mt_train),
#     batch_size=batch_size,
#     sort_within_batch=True,
#     sort_key=lambda x: len(x.src),
#     device=device)

train_iterator = data.BucketIterator(
     dataset=mt_train, batch_size=batch_size,
     sort_key=lambda x: data.interleave_keys(len(x.src), len(x.trg)), device=device)


In [14]:
encoder_net = Encoder(
    input_size_encoder, encoder_embedding_size, hidden_size, num_layers, enc_dropout
).to(device)


In [16]:
decoder_net = Decoder(
    input_size_decoder,
    decoder_embedding_size,
    hidden_size,
    output_size,
    num_layers,
    dec_dropout,
).to(device)

In [17]:
model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = 1
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


In [18]:
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

In [19]:
def translate_sentence(model, sentence, german, english, device, max_length=50):
    # Load german tokenizer
#     spacy_ger = spacy.load("de")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
#         tokens = [token.text.lower() for token in spacy_ger(sentence)]
        tokens = [i.lower() for i in tokenize(sentence, "hi")]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        outputs_encoder, hiddens, cells = model.encoder(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hiddens, cells = model.decoder(
                previous_word, outputs_encoder, hiddens, cells
            )
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence[1:]


def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [20]:
sentence = "प्रधानमंत्री ने कहा कि भारत में केंद्र सरकार बुनियादी ढांचे पर ध्यान केंद्रित कर रही है।"

In [21]:
# import pandas as pd

# df = pd.read_csv('./data_torch/train.csv')

# df.columns

# # df['hi_len'] = 
# sum(df['english_sentence'].apply(lambda x : len(x)) <=40)

# df = pd.read_excel('./data_torch/autocat_data.xlsx')

# df.shape

# sum(df['title'].apply(lambda x : len(x)) <=60)

In [22]:
for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

    model.eval()

    translated_sentence = translate_sentence(
        model, sentence, hindi, english, device, max_length=50
    )

    print(f"Translated example sentence: \n {translated_sentence}")

    model.train()
    
    for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # Forward prop
        output = model(inp_data, target)

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin. While we're at it
        # Let's also remove the start token while we're at it
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)

        # Back prop
        loss.backward()

        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # Plot to tensorboard
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1    

[Epoch 0 / 200]
=> Saving checkpoint
Translated example sentence: 
 ['democracy', '44', 'prime', 'its', 'department', 'india', 'has', 'india', 'yojana', 'with', 'being', 'funding', 'queen', 'committee', 'ministry', 'india', 'yojana', 'with', 'being', 'funding', 'queen', 'committee', 'ministry', 'india', 'yojana', 'with', 'being', 'funding', 'queen', 'committee', 'ministry', 'india', 'yojana', 'with', 'being', 'funding', 'queen', 'committee', 'ministry', 'india', 'yojana', 'with', 'being', 'funding', 'queen', 'committee', 'ministry', 'india', 'yojana', 'with']
[Epoch 1 / 200]
=> Saving checkpoint
Translated example sentence: 
 ['<unk>']
[Epoch 2 / 200]
=> Saving checkpoint
Translated example sentence: 
 ['<unk>']
[Epoch 3 / 200]
=> Saving checkpoint
Translated example sentence: 
 ['<unk>']
[Epoch 4 / 200]
=> Saving checkpoint
Translated example sentence: 
 ['<unk>']
[Epoch 5 / 200]
=> Saving checkpoint
Translated example sentence: 
 ['<unk>']
[Epoch 6 / 200]
=> Saving checkpoint


KeyboardInterrupt: 

In [23]:
import pandas as pd

In [24]:
df = pd.read_csv('./data_torch/pmindia.v1.hi-en.csv')

In [26]:
tokenize_hi(df['hindi_sentence'][100])

['▁प्रधानमंत्री',
 '▁ने',
 '▁कहा',
 '▁कि',
 '▁भारत',
 '▁में',
 '▁केंद्र',
 '▁सरकार',
 '▁बुनियादी',
 '▁ढांचे',
 '▁पर',
 '▁ध्यान',
 '▁केंद्रित',
 '▁कर',
 '▁रही',
 '▁है',
 '']

In [62]:
type(sentence)

str